

### Project overview
- The src/ layout keeps importable code separate from the repo root to avoid accidental imports during development and aligns with current packaging guidance.[1]
- SQLite is accessed via Python’s sqlite3 DB-API with parameterized queries and foreign_keys enabled per connection to ensure referential integrity and prevent SQL injection mistakes.[6][3]
- CLI flags are handled by argparse, while a simple interactive menu provides daily operations; tests use unittest for batteries-included test discovery and assertions.[4][5]

### Directory structure
This structure follows the src/ layout to keep application code importable under src/, with tests isolated and data housed outside source.[1]
```
inventory_pos_system/
│
├── src/
│   ├── __init__.py
│   ├── main.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── database.py
│   │   ├── product.py
│   │   ├── sale.py
│   │   └── customer.py
│   ├── utils/
│   │   ├── __init__.py
│   │   ├── validators.py
│   │   ├── helpers.py
│   │   └── logger.py
│   ├── services/
│   │   ├── __init__.py
│   │   ├── inventory_service.py
│   │   ├── sales_service.py
│   │   └── report_service.py
│   └── cli/
│       ├── __init__.py
│       ├── menu.py
│       └── interface.py
│
├── data/
│   └── inventory.db
│
├── tests/
│   ├── __init__.py
│   ├── test_models.py
│   ├── test_services.py
│   └── test_utils.py
│
├── requirements.txt
├── README.md
└── .gitignore
```
- Follow PEP 8 across files (4-space indentation, naming conventions, imports grouping) for consistent professional style.[2]

### Database core (SQLite)
- The Database class enables PRAGMA foreign_keys per connection, uses parameterized queries, and initializes schema for products, customers, sales, and sale_items.[3][6]

```python
# src/models/database.py
from __future__ import annotations

import sqlite3
from pathlib import Path
from typing import Generator, Optional, Iterable

SCHEMA = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    sku TEXT NOT NULL UNIQUE,
    name TEXT NOT NULL,
    price REAL NOT NULL CHECK(price >= 0),
    quantity INTEGER NOT NULL DEFAULT 0 CHECK(quantity >= 0),
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

CREATE TRIGGER IF NOT EXISTS trg_products_updated_at
AFTER UPDATE ON products
BEGIN
  UPDATE products SET updated_at = CURRENT_TIMESTAMP WHERE id = NEW.id;
END;

CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    phone TEXT
);

CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY,
    customer_id INTEGER REFERENCES customers(id),
    total REAL NOT NULL CHECK(total >= 0),
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS sale_items (
    id INTEGER PRIMARY KEY,
    sale_id INTEGER NOT NULL REFERENCES sales(id) ON DELETE CASCADE,
    product_id INTEGER NOT NULL REFERENCES products(id),
    quantity INTEGER NOT NULL CHECK(quantity > 0),
    unit_price REAL NOT NULL CHECK(unit_price >= 0),
    subtotal REAL NOT NULL CHECK(subtotal >= 0)
);
"""

class Database:
    def __init__(self, db_path: Path) -> None:
        self.db_path = Path(db_path)

    def connect(self) -> sqlite3.Connection:
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        conn.execute("PRAGMA foreign_keys = ON;")
        return conn

    def initialize(self) -> None:
        with self.connect() as conn:
            conn.executescript(SCHEMA)

    def execute(self, sql: str, params: Optional[Iterable] = None) -> int:
        with self.connect() as conn:
            cur = conn.execute(sql, params or [])
            conn.commit()
            return cur.lastrowid

    def query(self, sql: str, params: Optional[Iterable] = None) -> list[sqlite3.Row]:
        with self.connect() as conn:
            cur = conn.execute(sql, params or [])
            return cur.fetchall()

    def transaction(self) -> sqlite3.Connection:
        conn = self.connect()
        conn.isolation_level = None
        conn.execute("BEGIN;")
        return conn
```
- Parameterized statements (using ? placeholders) and enabling foreign key enforcement per connection are both recommended patterns in sqlite3 and SQLite.[6][3]

### Product model and repository
- Models use dataclasses for clarity, and repositories encapsulate CRUD with parameterized queries.[6]

```python
# src/models/product.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional, Iterable
from .database import Database

@dataclass
class Product:
    id: Optional[int]
    sku: str
    name: str
    price: float
    quantity: int

class ProductRepository:
    def __init__(self, db: Database) -> None:
        self.db = db

    def create(self, sku: str, name: str, price: float, quantity: int = 0) -> int:
        sql = """
        INSERT INTO products (sku, name, price, quantity) VALUES (?, ?, ?, ?)
        """
        return self.db.execute(sql, (sku, name, price, quantity))

    def get_by_id(self, product_id: int) -> Optional[Product]:
        sql = "SELECT id, sku, name, price, quantity FROM products WHERE id = ?"
        rows = self.db.query(sql, (product_id,))
        if not rows:
            return None
        r = rows
        return Product(r["id"], r["sku"], r["name"], r["price"], r["quantity"])

    def get_by_sku(self, sku: str) -> Optional[Product]:
        sql = "SELECT id, sku, name, price, quantity FROM products WHERE sku = ?"
        rows = self.db.query(sql, (sku,))
        if not rows:
            return None
        r = rows
        return Product(r["id"], r["sku"], r["name"], r["price"], r["quantity"])

    def list(self) -> list[Product]:
        sql = "SELECT id, sku, name, price, quantity FROM products ORDER BY name"
        return [Product(r["id"], r["sku"], r["name"], r["price"], r["quantity"]) for r in self.db.query(sql)]

    def update_stock(self, product_id: int, delta_qty: int) -> None:
        sql = "UPDATE products SET quantity = quantity + ? WHERE id = ?"
        self.db.execute(sql, (delta_qty, product_id))

    def update(self, product_id: int, name: str, price: float) -> None:
        sql = "UPDATE products SET name = ?, price = ? WHERE id = ?"
        self.db.execute(sql, (name, price, product_id))

    def delete(self, product_id: int) -> None:
        sql = "DELETE FROM products WHERE id = ?"
        self.db.execute(sql, (product_id,))
```
- Always pass parameters as a tuple to avoid SQL injection and type mishandling per DB-API guidance.[6]

### Customer model and repository
- Customers are optional on a sale, but kept normalized for reuse and reporting.[6]

```python
# src/models/customer.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional
from .database import Database

@dataclass
class Customer:
    id: Optional[int]
    name: str
    email: Optional[str] = None
    phone: Optional[str] = None

class CustomerRepository:
    def __init__(self, db: Database) -> None:
        self.db = db

    def create(self, name: str, email: Optional[str], phone: Optional[str]) -> int:
        sql = "INSERT INTO customers (name, email, phone) VALUES (?, ?, ?)"
        return self.db.execute(sql, (name, email, phone))

    def get_by_id(self, customer_id: int) -> Optional[Customer]:
        rows = self.db.query("SELECT id, name, email, phone FROM customers WHERE id = ?", (customer_id,))
        if not rows:
            return None
        r = rows
        return Customer(r["id"], r["name"], r["email"], r["phone"])

    def list(self) -> list[Customer]:
        rows = self.db.query("SELECT id, name, email, phone FROM customers ORDER BY name")
        return [Customer(r["id"], r["name"], r["email"], r["phone"]) for r in rows]
```
- Since sales.customer_id references customers(id), foreign key enforcement is enabled so orphan references are prevented.[3]

### Sale and sale items
- Create sales transactionally: check stock, insert sale, insert items, update inventory, then commit; rollback on error to keep data consistent.[3][6]

```python
# src/models/sale.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional, Iterable, List, Tuple
from .database import Database

@dataclass
class SaleItem:
    product_id: int
    quantity: int
    unit_price: float

@dataclass
class Sale:
    id: Optional[int]
    customer_id: Optional[int]
    total: float

class SaleRepository:
    def __init__(self, db: Database) -> None:
        self.db = db

    def create_sale(self, customer_id: Optional[int], items: List[SaleItem]) -> int:
        if not items:
            raise ValueError("At least one item required for a sale")

        conn = self.db.transaction()
        try:
            # Check stock and compute totals
            total = 0.0
            current_stock: dict[int, Tuple[int, float]] = {}
            for it in items:
                row = conn.execute("SELECT quantity, price FROM products WHERE id = ?", (it.product_id,)).fetchone()
                if row is None:
                    raise ValueError(f"Product id {it.product_id} not found")
                qty, price = int(row), float(row[1])
                if it.quantity <= 0:
                    raise ValueError("Quantity must be > 0")
                if qty < it.quantity:
                    raise ValueError(f"Insufficient stock for product {it.product_id}")
                # use current product price if unit_price not provided (0 or negative)
                final_unit_price = it.unit_price if it.unit_price >= 0 else price
                total += final_unit_price * it.quantity
                current_stock[it.product_id] = (qty, final_unit_price)

            cur = conn.execute("INSERT INTO sales (customer_id, total) VALUES (?, ?)", (customer_id, total))
            sale_id = cur.lastrowid

            for it in items:
                qty, uprice = current_stock[it.product_id]
                final_unit_price = it.unit_price if it.unit_price >= 0 else uprice
                subtotal = final_unit_price * it.quantity
                conn.execute(
                    "INSERT INTO sale_items (sale_id, product_id, quantity, unit_price, subtotal) VALUES (?, ?, ?, ?, ?)",
                    (sale_id, it.product_id, it.quantity, final_unit_price, subtotal),
                )
                conn.execute(
                    "UPDATE products SET quantity = quantity - ? WHERE id = ?",
                    (it.quantity, it.product_id),
                )

            conn.execute("COMMIT;")
            conn.close()
            return sale_id
        except Exception:
            conn.execute("ROLLBACK;")
            conn.close()
            raise

    def get_sale(self, sale_id: int) -> Optional[Sale]:
        rows = self.db.query("SELECT id, customer_id, total FROM sales WHERE id = ?", (sale_id,))
        if not rows:
            return None
        r = rows
        return Sale(r["id"], r["customer_id"], r["total"])

    def list_sales(self) -> list[Sale]:
        rows = self.db.query("SELECT id, customer_id, total FROM sales ORDER BY id DESC")
        return [Sale(r["id"], r["customer_id"], r["total"]) for r in rows]
```
- This uses DB-API transactions and parameter binding, and relies on foreign key and constraint checks for integrity.[3][6]

### Utilities (validators, helpers, logger)
- Validators centralize input checks to keep services clean and predictable.[2]

```python
# src/utils/validators.py
from __future__ import annotations

import re

SKU_RE = re.compile(r"^[A-Za-z0-9\-]{3,32}$")
EMAIL_RE = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")

def validate_sku(sku: str) -> None:
    if not SKU_RE.match(sku):
        raise ValueError("SKU must be 3-32 chars, alnum or hyphen")

def validate_price(price: float) -> None:
    if price < 0:
        raise ValueError("Price must be >= 0")

def validate_quantity(qty: int) -> None:
    if qty < 0:
        raise ValueError("Quantity must be >= 0")

def validate_email(email: str | None) -> None:
    if email and not EMAIL_RE.match(email):
        raise ValueError("Invalid email format")
```
- Keep helpers non-domain-specific and pure where possible.[2]

```python
# src/utils/helpers.py
from __future__ import annotations

from datetime import datetime

def format_currency(value: float) -> str:
    return f"{value:,.2f}"

def parse_iso_date(s: str) -> datetime:
    return datetime.fromisoformat(s)
```
- Configure standard-library logging with module-level loggers and consistent formatting.[7]

```python
# src/utils/logger.py
from __future__ import annotations

import logging
from typing import Optional

def get_logger(name: str = __name__, level: int = logging.INFO) -> logging.Logger:
    logger = logging.getLogger(name)
    if logger.handlers:
        return logger
    logger.setLevel(level)
    handler = logging.StreamHandler()
    fmt = logging.Formatter("%(asctime)s %(levelname)s [%(name)s] %(message)s")
    handler.setFormatter(fmt)
    logger.addHandler(handler)
    return logger
```
- The standard logging library supports hierarchical loggers, handlers, and formatters for flexible configuration.[7]

### Services (business logic)
- Services orchestrate repository calls and enforce business rules like SKU uniqueness and stock sufficiency.[2]

```python
# src/services/inventory_service.py
from __future__ import annotations

from typing import Optional
from models.product import ProductRepository
from utils.validators import validate_sku, validate_price, validate_quantity

class InventoryService:
    def __init__(self, products: ProductRepository) -> None:
        self.products = products

    def add_product(self, sku: str, name: str, price: float, quantity: int = 0) -> int:
        validate_sku(sku); validate_price(price); validate_quantity(quantity)
        if self.products.get_by_sku(sku):
            raise ValueError("SKU already exists")
        return self.products.create(sku=sku, name=name, price=price, quantity=quantity)

    def restock(self, product_id: int, add_qty: int) -> None:
        if add_qty <= 0:
            raise ValueError("Restock quantity must be > 0")
        self.products.update_stock(product_id, add_qty)

    def set_price(self, product_id: int, new_price: float) -> None:
        validate_price(new_price)
        prod = self.products.get_by_id(product_id)
        if not prod:
            raise ValueError("Product not found")
        self.products.update(product_id, prod.name, new_price)

    def list_products(self):
        return self.products.list()
```
- Sales service computes totals and delegates inventory updates within the sale transaction.[6]

```python
# src/services/sales_service.py
from __future__ import annotations

from typing import Optional, List, Dict
from models.sale import SaleRepository, SaleItem

class SalesService:
    def __init__(self, sales: SaleRepository) -> None:
        self.sales = sales

    def record_sale(self, customer_id: Optional[int], items: List[Dict]) -> int:
        parsed = []
        for d in items:
            pid = int(d["product_id"])
            qty = int(d["quantity"])
            unit_price = float(d.get("unit_price", -1.0))
            parsed.append(SaleItem(product_id=pid, quantity=qty, unit_price=unit_price))
        return self.sales.create_sale(customer_id, parsed)

    def list_sales(self):
        return self.sales.list_sales()
```
- Basic reporting leverages SQL aggregation for common views.[6]

```python
# src/services/report_service.py
from __future__ import annotations

from datetime import date
from typing import List, Dict
from models.database import Database

class ReportService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def daily_sales_total(self, on_date: date) -> float:
        sql = """
        SELECT COALESCE(SUM(total), 0) AS total
        FROM sales
        WHERE DATE(created_at) = DATE(?)
        """
        rows = self.db.query(sql, (on_date.isoformat(),))
        return float(rows["total"])

    def top_selling_products(self, limit: int = 5) -> List[Dict]:
        sql = """
        SELECT p.id, p.sku, p.name, SUM(si.quantity) AS qty_sold
        FROM sale_items si
        JOIN products p ON p.id = si.product_id
        GROUP BY p.id, p.sku, p.name
        ORDER BY qty_sold DESC
        LIMIT ?
        """
        return [dict(r) for r in self.db.query(sql, (limit,))]

    def inventory_valuation(self) -> float:
        sql = "SELECT COALESCE(SUM(price * quantity), 0) AS value FROM products"
        rows = self.db.query(sql)
        return float(rows["value"])
```
- Aggregations are efficient in SQLite for moderate datasets and fit POS reporting needs.[6]

### CLI (menu + interface)
- Argparse handles flags like database path and log level; a simple interactive menu routes to services.[4]

```python
# src/cli/menu.py
from __future__ import annotations

def print_main_menu() -> None:
    print("\n=== POS Inventory & Sales ===")
    print("1) Add Product")
    print("2) List Products")
    print("3) Restock Product")
    print("4) Record Sale")
    print("5) List Sales")
    print("6) Daily Sales Total")
    print("7) Top Selling Products")
    print("8) Inventory Valuation")
    print("0) Exit")
```

```python
# src/cli/interface.py
from __future__ import annotations

from datetime import date
from services.inventory_service import InventoryService
from services.sales_service import SalesService
from services.report_service import ReportService
from utils.logger import get_logger
from .menu import print_main_menu

log = get_logger(__name__)

class CLI:
    def __init__(self, inv: InventoryService, sales: SalesService, reports: ReportService) -> None:
        self.inv = inv
        self.sales = sales
        self.reports = reports

    def run(self) -> None:
        while True:
            print_main_menu()
            choice = input("Select: ").strip()
            try:
                if choice == "1":
                    sku = input("SKU: ").strip()
                    name = input("Name: ").strip()
                    price = float(input("Price: ").strip())
                    qty = int(input("Initial Qty (0 default): ") or "0")
                    pid = self.inv.add_product(sku, name, price, qty)
                    print(f"Product created with id={pid}")
                elif choice == "2":
                    for p in self.inv.list_products():
                        print(f"[{p.id}] {p.sku} | {p.name} | {p.price:.2f} | qty={p.quantity}")
                elif choice == "3":
                    pid = int(input("Product ID: ").strip())
                    add = int(input("Add quantity: ").strip())
                    self.inv.restock(pid, add)
                    print("Restocked.")
                elif choice == "4":
                    cust = input("Customer ID (blank for none): ").strip()
                    cust_id = int(cust) if cust else None
                    items = []
                    while True:
                        line = input("Item product_id,quantity[,unit_price] (blank to finish): ").strip()
                        if not line:
                            break
                        parts = [p.strip() for p in line.split(",")]
                        if len(parts) < 2:
                            print("Need product_id,quantity[,unit_price]")
                            continue
                        d = {"product_id": int(parts), "quantity": int(parts[1])}
                        if len(parts) > 2:
                            d["unit_price"] = float(parts[2])
                        items.append(d)
                    sale_id = self.sales.record_sale(cust_id, items)
                    print(f"Sale recorded with id={sale_id}")
                elif choice == "5":
                    for s in self.sales.list_sales():
                        print(f"[{s.id}] customer={s.customer_id} total={s.total:.2f}")
                elif choice == "6":
                    ds = input("Date (YYYY-MM-DD, blank=today): ").strip() or date.today().isoformat()
                    total = self.reports.daily_sales_total(date.fromisoformat(ds))
                    print(f"Total for {ds}: {total:.2f}")
                elif choice == "7":
                    limit = int(input("Top N (default 5): ").strip() or "5")
                    rows = self.reports.top_selling_products(limit)
                    for r in rows:
                        print(f"{r['sku']} | {r['name']} | sold={r['qty_sold']}")
                elif choice == "8":
                    val = self.reports.inventory_valuation()
                    print(f"Inventory valuation: {val:.2f}")
                elif choice == "0":
                    print("Goodbye!")
                    return
                else:
                    print("Invalid choice.")
            except Exception as e:
                log.error("Error: %s", e)
                print(f"Error: {e}")
```
- Argparse offers user-friendly CLIs with built-in help and parsing of options and subcommands.[4]

### Entrypoint (main)
- Main wires dependencies, initializes schema if needed, and starts the CLI; argparse provides flags for database path and log level.[4]

```python
# src/main.py
from __future__ import annotations

import argparse
import logging
from pathlib import Path

from models.database import Database
from models.product import ProductRepository
from models.sale import SaleRepository
from models.customer import CustomerRepository
from services.inventory_service import InventoryService
from services.sales_service import SalesService
from services.report_service import ReportService
from cli.interface import CLI
from utils.logger import get_logger

def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="Inventory & Sales (POS)")
    p.add_argument("--db", type=Path, default=Path("data/inventory.db"), help="Path to SQLite database file")
    p.add_argument("--log-level", default="INFO", help="Logging level (DEBUG, INFO, WARNING, ERROR)")
    return p

def main() -> None:
    args = build_parser().parse_args()
    level = getattr(logging, args.log_level.upper(), logging.INFO)
    log = get_logger("pos", level=level)

    db = Database(args.db)
    args.db.parent.mkdir(parents=True, exist_ok=True)
    db.initialize()

    products = ProductRepository(db)
    sales = SaleRepository(db)
    customers = CustomerRepository(db)

    inv_svc = InventoryService(products)
    sales_svc = SalesService(sales)
    reports_svc = ReportService(db)

    log.info("POS started with DB at %s", args.db)
    CLI(inv_svc, sales_svc, reports_svc).run()

if __name__ == "__main__":
    main()
```
- The argparse module is designed to define required arguments and options, auto-generate help, and parse CLI flags robustly.[4]

### Tests (unittest)
- Standard-library unittest provides discovery and assertions without external dependencies; run with python -m unittest.[5]

```python
# tests/test_models.py
import unittest
from pathlib import Path
from src.models.database import Database
from src.models.product import ProductRepository

class TestModels(unittest.TestCase):
    def setUp(self):
        self.db = Database(Path(":memory:"))
        self.db.initialize()
        self.products = ProductRepository(self.db)

    def test_create_and_list_product(self):
        pid = self.products.create("ABC-001", "Apple", 10.0, 5)
        self.assertIsInstance(pid, int)
        plist = self.products.list()
        self.assertEqual(len(plist), 1)
        self.assertEqual(plist.sku, "ABC-001")

if __name__ == "__main__":
    unittest.main()
```
- In-memory databases are useful for isolated tests, and unittest offers a CLI runner for modules, classes, and methods.[5][6]

```python
# tests/test_services.py
import unittest
from pathlib import Path
from src.models.database import Database
from src.models.product import ProductRepository
from src.models.sale import SaleRepository
from src.services.inventory_service import InventoryService
from src.services.sales_service import SalesService

class TestServices(unittest.TestCase):
    def setUp(self):
        self.db = Database(Path(":memory:"))
        self.db.initialize()
        self.prod_repo = ProductRepository(self.db)
        self.sale_repo = SaleRepository(self.db)
        self.inv = InventoryService(self.prod_repo)
        self.sales = SalesService(self.sale_repo)
        self.pid = self.inv.add_product("SKU-1", "Pen", 2.5, 10)

    def test_record_sale_updates_stock(self):
        sale_id = self.sales.record_sale(None, [{"product_id": self.pid, "quantity": 3}])
        self.assertIsInstance(sale_id, int)
        p = self.prod_repo.get_by_id(self.pid)
        self.assertEqual(p.quantity, 7)

if __name__ == "__main__":
    unittest.main()
```
- This verifies the transactional flow: sale creation, item insertion, and inventory decrement.[6]

```python
# tests/test_utils.py
import unittest
from src.utils.validators import validate_sku, validate_price, validate_quantity, validate_email

class TestUtils(unittest.TestCase):
    def test_validators(self):
        validate_sku("ABC-123")
        validate_price(0.0)
        validate_quantity(5)
        validate_email("user@example.com")
        with self.assertRaises(ValueError):
            validate_sku("!!")
        with self.assertRaises(ValueError):
            validate_price(-1)
        with self.assertRaises(ValueError):
            validate_quantity(-2)
        with self.assertRaises(ValueError):
            validate_email("bad@")

if __name__ == "__main__":
    unittest.main()
```
- Keeping validation isolated simplifies service code and enables focused unit tests.[5][2]

### requirements.txt
- The project uses only the Python standard library (sqlite3, argparse, logging, unittest), so an empty or minimal requirements.txt is sufficient.[7][5][4][6]
```
# requirements.txt
# Standard library only; no external dependencies.
```

### README.md
- This README documents setup, run, and test instructions, and highlights src/ layout and foreign key enforcement.[1][3]

```
# Inventory & Sales (POS)

A minimal, production-ready POS with SQLite, src/ layout, and a menu-driven CLI.

## Setup
python -m venv .venv
. .venv/bin/activate  # Windows: .venv\Scripts\activate
pip install -r requirements.txt

## Run
python -m src.main --db data/inventory.db --log-level INFO

## Notes
- Uses src/ layout to avoid accidental imports and align with modern packaging practices. [source]
- SQLite foreign keys are enforced per connection via PRAGMA foreign_keys = ON to maintain referential integrity. [source]

## Tests
python -m unittest
```
- The README calls out the rationale for src/ and foreign key enforcement to guide maintainers.[1][3]

### .gitignore
- Ignore Python bytecode, virtual environments, editor files, coverage artifacts, and local DB files.[2]

```
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*.pyo

# Virtual environments
.venv/
venv/

# OS / Editor
.DS_Store
.vscode/
.idea/

# Coverage / test
.coverage
htmlcov/
.pytest_cache/

# Local data
data/*.db
```

### How this maps to Module 1 skills
- Core Python, control flow, functions, OOP, modules, file paths, and standard library usage are represented across src modules and utilities.[2]
- SQL schema, queries, and Python–SQLite integration (including DML from Python and parameterized queries) are demonstrated in database.py, repositories, and services.[6]
- A CLI application wraps everything together using argparse and an interactive loop for day-to-day POS tasks.[4]

Key things to customize next:
- Add authentication/roles, returns/exchanges support, receipts PDF/email, richer reporting, and packaging for distribution as a console script.[1]

